In [1]:
import networkx as nx
import os
import random
import matplotlib.pyplot as plt
import math
from itertools import permutations
from itertools import combinations
from itertools import chain
import numpy as np
from networkx.algorithms.matching import max_weight_matching
from networkx.algorithms.euler import eulerian_circuit
import pandas as pd

def compute_cycle_length(graph,cycle) :
    """This function takes a graph instance and a cycle and returns the length of this cycle.  """
    """graph : networkx instance"""
    """cycle : tuple and in some cases can be passed as a list. """
    length = 0
    for vertex_id in range(len(cycle)) :
        length += graph[cycle[vertex_id]][cycle[(vertex_id + 1) % len(cycle)]]['weight']
    return length

def double_approximation(graph,source_vertex = 0) :
    """This function implements the double approximation algorithm to solve Metric TSP."""
    #1.Compute the MST for the inputed graph .
    mst = nx.minimum_spanning_tree(graph) 
    #2.Traverse the mst .
    cycle = nx.dfs_preorder_nodes(mst,source_vertex)
    cycle = tuple(cycle)
    weight = compute_cycle_length(graph,cycle)
    return cycle , weight

def christofides(graph,source_vertex = 0) :
    """
    This function is an implementation of Christofides algorithm .
    graph : networkx instance .
    source_vertex : The starting and ending point of the cycle .
    """
    #1.compute MST .
    mst = nx.minimum_spanning_tree(graph)
    #2.find the vertices with odd degrees .
    odd_degree_vertices = list()
    for vertex in graph.nodes() :
        if(mst.degree(vertex) % 2 != 0) :
            odd_degree_vertices.append(vertex)
    #3.build subgraph with only vertices with odd degrees and the edges between them .
    odd_vertices_graph = graph.subgraph(odd_degree_vertices)
    #4.compute minimum weight perfect matching on the odd vertices subgraph .
    for edge in odd_vertices_graph.edges() :
        odd_vertices_graph[edge[0]][edge[1]]['weight'] = -1 * odd_vertices_graph[edge[0]][edge[1]]['weight']
    minimum_weight_perfect_matching = list(max_weight_matching(odd_vertices_graph, maxcardinality=True))
    #To correct negative edges .
    for edge in graph.edges() :
        graph[edge[0]][edge[1]]['weight'] = abs(graph[edge[0]][edge[1]]['weight'])
    #5.combine the mst with the edges from minimum weight perfect matching .
    multi_graph = nx.MultiGraph(mst)
    for edge in minimum_weight_perfect_matching :
        multi_graph.add_edge(edge[0],edge[1],weight = graph[edge[0]][edge[1]]['weight'])
    #6.find Euler's cycle
    euler_cycle = list(eulerian_circuit(multi_graph,source = source_vertex))
    cycle = list()
    for edge in euler_cycle :
        if(edge[0] not in cycle) :
            cycle.append(edge[0])
        if(edge[1] not in cycle) :
            cycle.append(edge[1])
    weight = compute_cycle_length(graph,tuple(cycle))
    return tuple(cycle) , weight

def generate_complete_undirected_weighted(num_vertices, min_weight= 1, max_weight= 10):
    graph = nx.Graph()
    for source in range(num_vertices):
        for destination in range(source):
            weight = random.randint(min_weight, max_weight)
            graph.add_edge(source, destination, weight= weight)
    return graph

def satisfy_triangel(graph):
    fw = nx.floyd_warshall(graph, weight='weight')
    output_graph = nx.Graph()
    for source, dictionary in fw.items():
        for destination, weight in dictionary.items():
            if(source != destination):
                output_graph.add_edge(source, destination, weight= weight)
    return output_graph

def generate_delta_tsp_input(num_vertices, min_weight= 1, max_weight= 10):
    graph = generate_complete_undirected_weighted(num_vertices, min_weight, max_weight)
    output_graph = satisfy_triangel(graph)
    return output_graph

def is_valid(g):
    n = len(g.nodes())
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if((i != j) and (i != k) and (j !=k)):
                    if(g[i][k]['weight'] + g[k][j]['weight'] < g[i][j]['weight']):
                        return False
    return True

In [2]:
g = generate_delta_tsp_input(6, 1, 4)
print(is_valid(g))
print(g.edges.data('weight'))

True
[(1, 0, 2), (1, 2, 3), (1, 3, 1), (1, 4, 2), (1, 5, 2), (0, 2, 2), (0, 3, 1), (0, 4, 1), (0, 5, 3), (2, 3, 2), (2, 4, 1), (2, 5, 1), (3, 4, 1), (3, 5, 3), (4, 5, 2)]


In [3]:
cycle, weight = double_approximation(g)
print(cycle, weight)

(0, 3, 1, 4, 2, 5) 9


In [4]:
cycle, weight = christofides(g)
print(cycle, weight)

(0, 4, 2, 5, 1, 3) 7


In [5]:
results = []
valid = 0
for i in range(100):
    g = generate_delta_tsp_input(25, 1, 100)
    if(is_valid(g)):
        valid +=1
    cycle1, weight1 = double_approximation(g)
    cycle2, weight2 = christofides(g)
    results.append((weight1, weight2))
res = [1 if weight1 >= weight2 else 0 for weight1, weight2 in results]
print("Valid input : ", round(valid,2),"%.")
print(f"Out of 100 times, Christofides gives better results(weights for cycles) than 2-approximation in {sum(res)} cases.")

Valid input :  100 %.
Out of 100 times, Christofides gives better results(weights for cycles) than 2-approximation in 100 cases.


In [6]:
# example to test
g = nx.Graph()
g.add_weighted_edges_from([(1,2,1),(0,1,1),(2,0,3)])
print(is_valid(g))
cycle1, weight1 = double_approximation(g)
cycle2, weight2 = christofides(g)
print(cycle1, weight1)
print(cycle2, weight2)

False
(0, 1, 2) 5
(0, 2, 1) 5
